In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
import astropy.constants as const
import astropy.units as u
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.modeling.models import Gaussian2D as AstropyGauss2d
from astropy.time import Time


from fastimgproto.sourcefind.fit import Gaussian2dParams
from fastimgproto.sourcefind.image import (
    SourceFindImage, IslandParams, PixelIndex, Pixel,)
from fastimgproto.bindings.sourcefind import cpp_sourcefind

from fastimgproto.fixtures.image import (
    add_gaussian2d_to_image,
    gaussian_point_source,
)
from fastimgproto.fixtures.sourcefind import random_sources_on_grid

In [ ]:
import stp_python

In [ ]:
image_size = 64
image_shape = (image_size, image_size)
# ydim = 64
# xdim = 32
# image_shape = (ydim, xdim)

In [ ]:
n_sources=1
amplitude_range = (28., 42.)
semiminor_range = (1.2, 2.5)
axis_ratio_range = (1., 2.)
seed = 123456

In [ ]:
sources_grid = random_sources_on_grid(image_size,
                                     n_sources,
                                     amplitude_range=amplitude_range,
                                     semiminor_range=semiminor_range,
                                     axis_ratio_range=axis_ratio_range,
                                     seed=seed
                                )

In [ ]:
tough_source = Gaussian2dParams(
    x_centre=18.48820770355941, y_centre=34.05960466345559, amplitude=12.609448034550804,
    semimajor=2.913573230793465, semiminor=0.7230940150989847, theta=-1.4507887213254516)

test_source = gaussian_point_source(18, 34.5, amplitude=18.3, semimajor=2.5, semiminor=1.5)
img = np.zeros(image_shape)
add_gaussian2d_to_image(tough_source, img)


# for s in sources_grid:
#     add_gaussian2d_to_image(s, img)

In [ ]:
sf_img = SourceFindImage(img, detection_n_sigma=5., analysis_n_sigma=3., rms_est=1.)
island0 = sf_img.islands[0].params

In [ ]:
sf_img.fit_islands()

In [ ]:
# Call source_find
results = cpp_sourcefind(img, detection_n_sigma=5., analysis_n_sigma=3., rms_est=1.)

In [ ]:
results[0]

In [ ]:
tough_source

In [ ]:
results[0]

In [ ]:
island0

In [ ]:
len(sources_grid)

In [ ]:
len(ceres_islands)

In [ ]:
plt.imshow(img)
plt.colorbar()

In [ ]:
sfimg = SourceFindImage(img, detection_n_sigma=detection_n_sigma, analysis_n_sigma=analysis_n_sigma, rms_est=rms_est)

In [ ]:
binary_map = sfimg.data > sfimg.analysis_n_sigma * sfimg.rms_est

In [ ]:
plt.imshow(binary_map)